-----------------------------------------------------------

1.导包

In [ ]:
pip install transformers
pip install seqeval

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/9d/2d/233c79d5b4e5ab1dbf111242299153f3caddddbb691219f363ad55ce783d/seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16186 sha256=143845e5f90e4f765377086454338c771e63cf08fb5a09c8bcfa5bb6e4864c0d
  Stored in directory: c:\users\17397\appdata\local\pip\cache\wheels\3a\1b\f3\e58d19d62452680b7ac2b49c36aef791806f6274bd6652e29b
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import Trainer, TrainingArguments,AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

2.加载数据集

In [3]:
dataset=load_dataset('dirtycomputer/ChnSentiCorp_htl_all',split='train')
dataset=dataset.filter(lambda x:x['review'] is not None)#过滤掉review为空的样本
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

3.划分数据集

In [4]:
dataset=dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

### 4.数据预处理

In [5]:
import torch
tokenizer=AutoTokenizer.from_pretrained('hfl/chinese-macbert-large')
def process_function(examples):
    tokenized_examples=tokenizer(examples['review'],max_length=128,truncation=True,padding='max_length')
    #max_length=32设置成32，可提升速度，但是会略微降低模型精度。
    tokenized_examples['label']=examples['label']
    return tokenized_examples
tokenized_dataset=dataset.map(process_function,batched=True,remove_columns=dataset['train'].column_names)
tokenized_dataset

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 777
    })
})

In [6]:
# from torch.utils.data import Dataset, DataLoader
# from transformers import DataCollatorWithPadding
# trainset,validset=tokenized_dataset['train'],tokenized_dataset['test']
# trainloader=DataLoader(trainset,batch_size=32,shuffle=True,collate_fn=DataCollatorWithPadding(tokenizer))
# validloader=DataLoader(validset,batch_size=64,shuffle=False,collate_fn=DataCollatorWithPadding(tokenizer))

### 5.创建模型及优化器

In [7]:
from torch.optim import Adam
model=AutoModelForSequenceClassification.from_pretrained('hfl/chinese-macbert-large')
# if torch.cuda.is_available():
#     model=model.cuda()
# trainer会自动判定是否使用cuda

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# optimizer=Adam(model.parameters(),lr=2e-5)

### 6.训练与验证

In [9]:
import evaluate
# clf_metrics=evaluate.combine(['accuracy', 'f1'])
acc_metrics=evaluate.load('accuracy')
f1_metrics=evaluate.load('f1')

In [10]:
# 2------------------
def evla_metrics(eval_predict):
    predictions,labels=eval_predict
    predictions=predictions.argmax(-1)
    acc=acc_metrics.compute(predictions=predictions,references=labels)
    f1=f1_metrics.compute(predictions=predictions,references=labels)
    acc.update(f1)
    return acc

### 7.创建TrainingArguments,2------------------------------

In [15]:
# 2***********************
train_args=TrainingArguments(output_dir='./checkpoints',                     #输出路径   
                             per_device_train_batch_size=32,                  #训练batch大小
                             per_device_eval_batch_size=32,                   #验证batch大小
                            #  gradient_accumulation_steps=32,                   #梯度累积步数
                            # gradient_checkpointing=True,                       #梯度检查点
                            # optim='adafactor',                              #优化器
                             num_train_epochs=1,                             #训练轮数
                             logging_steps=10,                              #日志记录步数
                             evaluation_strategy='epoch',                    #验证策略
                             save_strategy='epoch',                         #保存策略
                             learning_rate=2e-5,                             #学习率
                             weight_decay=0.01,                              #权重衰减
                             metric_for_best_model='f1',                #最好的模型指标
                             save_total_limit=3,                            #保存模型数量
                             load_best_model_at_end=True,                   #是否加载最好的模型
                             )
train_args

D:\Environment\Python\python3.10\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsd

In [12]:
# 2--------------------------创建trainer
from transformers import DataCollatorWithPadding

for name,param in model.bert.named_parameters():# 冻结bert的参数，只训练classifier层
    param.requires_grad=False

trainer=Trainer(model=model,
                args=train_args,
                train_dataset=tokenized_dataset['train'],
                eval_dataset=tokenized_dataset['test'],
                data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                compute_metrics=evla_metrics
                )

### 8. 模型训练2------------------

In [13]:
# 2-------------------
trainer.train()

  0%|          | 0/219 [00:00<?, ?it/s]

{'loss': 0.6457, 'grad_norm': 7.392805099487305, 'learning_rate': 1.9086757990867582e-05, 'epoch': 0.05}
{'loss': 0.4691, 'grad_norm': 4.017070293426514, 'learning_rate': 1.8173515981735163e-05, 'epoch': 0.09}
{'loss': 0.288, 'grad_norm': 6.901145935058594, 'learning_rate': 1.726027397260274e-05, 'epoch': 0.14}
{'loss': 0.2731, 'grad_norm': 5.2855048179626465, 'learning_rate': 1.634703196347032e-05, 'epoch': 0.18}
{'loss': 0.2634, 'grad_norm': 8.173861503601074, 'learning_rate': 1.54337899543379e-05, 'epoch': 0.23}
{'loss': 0.2719, 'grad_norm': 8.717487335205078, 'learning_rate': 1.4520547945205482e-05, 'epoch': 0.27}
{'loss': 0.2362, 'grad_norm': 3.509847402572632, 'learning_rate': 1.360730593607306e-05, 'epoch': 0.32}
{'loss': 0.2901, 'grad_norm': 8.262314796447754, 'learning_rate': 1.2694063926940641e-05, 'epoch': 0.37}
{'loss': 0.2912, 'grad_norm': 3.148289680480957, 'learning_rate': 1.178082191780822e-05, 'epoch': 0.41}
{'loss': 0.2331, 'grad_norm': 4.217838287353516, 'learning_ra

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.21408721804618835, 'eval_accuracy': 0.9111969111969112, 'eval_f1': 0.9362880886426593, 'eval_runtime': 12.2132, 'eval_samples_per_second': 63.62, 'eval_steps_per_second': 2.047, 'epoch': 1.0}
{'train_runtime': 8597.3077, 'train_samples_per_second': 0.813, 'train_steps_per_second': 0.025, 'train_loss': 0.2771923482145893, 'epoch': 1.0}


TrainOutput(global_step=219, training_loss=0.2771923482145893, metrics={'train_runtime': 8597.3077, 'train_samples_per_second': 0.813, 'train_steps_per_second': 0.025, 'total_flos': 1628084091734016.0, 'train_loss': 0.2771923482145893, 'epoch': 1.0})

### 9.模型评估

In [14]:
trainer.evaluate(tokenized_dataset['test'])

  0%|          | 0/25 [00:00<?, ?it/s]

FailedPreconditionError: ./checkpoints\runs\Jan09_22-22-06_周立庆 is not a directory

### 10.模型预测

In [ ]:
trainer.predict(tokenized_dataset['test'])

In [14]:
#原来的


SyntaxError: unexpected character after line continuation character (1406375738.py, line 2)